# Chapter 6: Rerandomization and Regression Adjustment

In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import statsmodels.api as sm
import statsmodels.formula.api as smf
# viz
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
font = {'family' : 'IBM Plex Sans Condensed',
               'weight' : 'normal',
               'size'   : 10}
plt.rc('font', **font)
plt.rcParams['figure.figsize'] = (6, 6)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


## Regression Adjustment

In [11]:
angrist = pd.read_stata("star.dta")
angrist2 = angrist.query("control == 1 | sfsp == 1")
angrist2['y'] = angrist2.GPA_year1.fillna(angrist2.GPA_year1.mean())


/tmp/ipykernel_129775/576707837.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angrist2['y'] = angrist2.GPA_year1.fillna(angrist2.GPA_year1.mean())


In [20]:
y, z, x = angrist2.y.values, angrist2.sfsp.values, angrist2.loc[:, ['female', 'gpa0']].values


### unadjusted regression

In [37]:
restable = (sm.OLS(y, sm.add_constant(z)).fit(cov_type='HC2').summary().tables[1].as_html())
unadj_res = pd.read_html(restable, header=0, index_col=0)[0].iloc[1, 0:4]


### adjusted (Lin 2013) regression

In [38]:
# standardize x
x = (x - x.mean(axis = 0))/x.std(axis = 0)


In [39]:
restable = (sm.OLS(y,
        sm.add_constant(
            np.c_[z, x, z[:,np.newaxis]*x]
            ))
    .fit(cov_type='HC2').summary().tables[1].as_html()
)
lin_res = pd.read_html(restable, header=0, index_col=0)[0].iloc[1,0:4]


In [43]:
pd.DataFrame(np.c_[unadj_res, lin_res].T,
    columns=['coef', 'se', 't', 'p'],
    index=['unadjusted', 'adjusted']
)


,coef,se,t,p
unadjusted,0.0518,0.078,0.669,0.504
adjusted,0.0682,0.074,0.925,0.355


## Rerandomization simulation

TBD

In [ ]:
def Mahalanobis2(z, x):
    x1 = x[z == 1, :]
    x0 = x[z == 0, :]
    n0, n1 = x0.shape[0], x1.shape[0]
